<a href="https://colab.research.google.com/github/cemvardar/pragmatic_google_colab/blob/main/dslab_colab_utility_chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install OpenAI
import openai
from google.colab import userdata
from time import sleep
import json
import time
from datetime import datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.7/328.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
def check_run(client, thread_id, run_id):
    while True:
        # Refresh the run object to get the latest status
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run_id
        )

        if run.status == "completed":
            print(f"Run is completed.")
            break
        elif run.status == "expired":
            print(f"Run is expired.")
            break
        else:
            print(f"OpenAI: Run is not yet completed. Waiting...{run.status}")
            time.sleep(10)  # Wait for 1 second before checking again

def get_file_name_for_export_with_date_time(file_name_header, file_extenstion):
    formatted_datetime = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    file_name = f"{file_name_header}_{formatted_datetime}.{file_extenstion}"
    file_name = file_name.replace(' ', '_')
    return file_name


def last_chatgpt_reponse(messages):
    return messages.data[0].content[0].text.value

class ChatGpt_Assitant():
    def __init__(self, name, instructions, tools, model):
        self.client = openai.Client(api_key=userdata.get('openai_api_key'))
        self.assistant = self.client.beta.assistants.create(
            name=name,
            instructions=instructions,
            tools=tools,
            model=model
        )
        self.thread = self.client.beta.threads.create()
        self.messages = None

    def add_user_input(self, user_input, print_repsonse=True):
        message = self.client.beta.threads.messages.create(thread_id=self.thread.id,
                                                    role="user",
                                                    content=user_input
        )

        run = self.client.beta.threads.runs.create(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id,
        )

        check_run(self.client, self.thread.id, run.id)

        self.messages = self.client.beta.threads.messages.list(
            thread_id=self.thread.id
        )


        assistant_message = self.messages.data[0].content[0].text.value
        if print_repsonse:
            print(len(self.messages.data))
            print(assistant_message)

        return last_chatgpt_reponse(self.messages)


def get_chatgpt_shopify_helper():
    name = "Shopify helper"
    instructions="""You are a shopify site content generator for products and product collections.
                    You generate  product titles and descriptions using the details provived by the users.
                    you follow instructions and feedback from the users to generate the best titles and descriptions
                    for maximizing product sales.
                    """
    tools=[{"type": "code_interpreter"}]
    model="gpt-4o"
    return ChatGpt_Assitant(name, instructions, tools, model)


def parse_json_from_gpt_response(message):
    start_index = message.find('```json') + 8
    end_index = message.rfind("```")
    json_content = json.loads(message[start_index:end_index].replace('\n\n', ''))
    return json_content


In [ ]:
def get_json_response_from_chat_gpt(prompt):
    gpt_assistant = get_chatgpt_shopify_helper()
    message = gpt_assistant.add_user_input(prompt, False)
    json_response = parse_json_from_gpt_response(message)
    return json_response

In [ ]:
# gpt_assistant = get_chatgpt_shopify_helper()
# gpt_assistant.add_user_input('Respond with a JSON object. Content for an example product', False)
# message

OpenAI: Run is not yet completed. Waiting...queued
Run is completed.


In [ ]:
prompt_title= """
Find a shopify product title and description including details about stone properties and
possible styles for the following product. title and description will be in english.
İnanna Lapis Kolye - Gold
{'*Ürünlerimiz el yapımıdır.', '*İnci ve Lapis taşı kullanılmıştır.', '925 Gümüş/Silver', '*Ürünlerimiz 925 ayar gümüştür.', '*Sarı Altın kaplama yapılmıştır.', "*Kolye zincir uzunluğu 41 cm'dir. 5 cm uzatma zinciri vardır."}
'
Respond with a JSON object. keys will be title and description.
don't use column in text
"""

In [ ]:
# gpt_assistant = get_chatgpt_shopify_helper()
# message = gpt_assistant.add_user_input(prompt_title, False)
# print(message)
# parse_json_from_gpt_response(message)

OpenAI: Run is not yet completed. Waiting...queued
Run is completed.
```json
{
  "title": "Inanna Lapis Necklace - Gold",
  "description": "Our products are handmade. Featuring pearl and lapis stones. Made with 925 sterling silver and coated with yellow gold. The chain length is 41 cm with an additional 5 cm extender. The lapis stone is known for its deep blue color and is believed to provide wisdom and truth. The pearls symbolize purity and elegance. This versatile necklace can be styled with both casual and formal outfits, adding a touch of sophistication and grace."
}
```


{'title': 'Inanna Lapis Necklace - Gold',
 'description': 'Our products are handmade. Featuring pearl and lapis stones. Made with 925 sterling silver and coated with yellow gold. The chain length is 41 cm with an additional 5 cm extender. The lapis stone is known for its deep blue color and is believed to provide wisdom and truth. The pearls symbolize purity and elegance. This versatile necklace can be styled with both casual and formal outfits, adding a touch of sophistication and grace.'}

In [ ]:
bp_prompt = '''
give 6 bullet point items in english for the following product
İnanna Lapis Kolye - Gold
{'*Ürünlerimiz el yapımıdır.', '*İnci ve Lapis taşı kullanılmıştır.', '925 Gümüş/Silver', '*Ürünlerimiz 925 ayar gümüştür.', '*Sarı Altın kaplama yapılmıştır.', "*Kolye zincir uzunluğu 41 cm'dir. 5 cm uzatma zinciri vardır."}
Respond with a JSON object. key will be items
don't use column in text
'''
# message = gpt_assistant.add_user_input(bp_prompt, False)
# print(message)
# parse_json_from_gpt_response(message)

OpenAI: Run is not yet completed. Waiting...queued
Run is completed.
```json
{
  "items": [
    "Handmade craftsmanship",
    "Incorporates pearl and lapis stones",
    "Constructed with 925 sterling silver",
    "Yellow gold plating",
    "Necklace chain length is 41 cm with 5 cm extender",
    "Lapis stone symbolizes wisdom and truth; pearls symbolize purity and elegance"
  ]
}
```


{'items': ['Handmade craftsmanship',
  'Incorporates pearl and lapis stones',
  'Constructed with 925 sterling silver',
  'Yellow gold plating',
  'Necklace chain length is 41 cm with 5 cm extender',
  'Lapis stone symbolizes wisdom and truth; pearls symbolize purity and elegance']}

In [ ]:
# message